In [1]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
from transformers.models.t5 import T5ForConditionalGeneration, T5Tokenizer

Load unlabeled data from CSV for inference

In [2]:
df = pd.read_csv('./data/unlabelled_df.csv')

In [3]:
df.head()

,submission_id,submission_title,submission_selftext,submission_link_flair_text,reply_body,all_text
0,aglcrj,Goodnotes 4 vs. Goodnotes 5 right now,I have used Goodnotes 4 for work a ton. And I...,NaN,"[""I'm getting a ton of bugs with 5 as well (sn...",Goodnotes 4 vs. Goodnotes 5 right now I have u...
1,agoowm,The bundle is available !,NaN,NaN,['Thank you. I have been waiting!\n\n&amp;#x20...,The bundle is available ! Thank you. I have b...
2,agpzxb,What happened to the pen (Goodnotes 5)?,I just got Goodnotes 5 and I was so excited fo...,NaN,"[""Have you tried the ball pen? That was the cl...",What happened to the pen (Goodnotes 5)? I just...
3,agq8qv,Non Apple Pencil styluses on GOodnotes 5?,I've been using a Wacom Bamboo stylus with Goo...,NaN,['According to the [review at Macstories](http...,Non Apple Pencil styluses on GOodnotes 5? I've...
4,agqksi,Text/typing in Goodnotes 5,"Notability user here, but trying out Goodnotes...",NaN,"[""That's neat af"", 'Seattle, hopefully not too...",Text/typing in Goodnotes 5 Notability user her...


In [4]:
all_strings = ('tag classification: ' + df['all_text']).to_list()

Load pretrained model and tokenizer

In [5]:
model = T5ForConditionalGeneration.from_pretrained("textomatic/subreddit-thread-tagging")
tokenizer = T5Tokenizer.from_pretrained("textomatic/subreddit-thread-tagging")

In [6]:
# To tokenize the list of strings before passing it to the t5 model
input_batch = tokenizer(
    all_strings,
    padding=True,
    truncation=True,
    return_tensors='pt',
    max_length=400
)
input_ids = input_batch["input_ids"]
attention_mask = input_batch["attention_mask"]

In [7]:
# Use the model to make prediction
all_preds = []
batch_size = 4

for i in range(0, len(all_strings), batch_size):
    print("Processing {}th data".format(i))
    subset_input_ids = input_ids[i:i+4]
    subset_attention_masks = attention_mask[i:i+4]

    if subset_input_ids.size()[0] >0:
        prediction = model.generate(
            input_ids=subset_input_ids,
            attention_mask=subset_attention_masks,
            num_beams=4,
            max_length=300,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1
        )

        for pred in prediction:
            theme = tokenizer.decode(pred).replace('<pad>','').replace('</s>','').strip()
            all_preds.append(theme)

Processing 0th data
Processing 4th data
Processing 8th data
Processing 12th data
Processing 16th data
Processing 20th data
Processing 24th data
Processing 28th data
Processing 32th data
Processing 36th data
Processing 40th data
Processing 44th data
Processing 48th data
Processing 52th data
Processing 56th data
Processing 60th data
Processing 64th data
Processing 68th data
Processing 72th data
Processing 76th data
Processing 80th data
Processing 84th data
Processing 88th data
Processing 92th data
Processing 96th data
Processing 100th data
Processing 104th data
Processing 108th data
Processing 112th data
Processing 116th data
Processing 120th data
Processing 124th data
Processing 128th data
Processing 132th data
Processing 136th data
Processing 140th data
Processing 144th data
Processing 148th data
Processing 152th data
Processing 156th data
Processing 160th data
Processing 164th data
Processing 168th data
Processing 172th data
Processing 176th data
Processing 180th data
Processing 184th

In [8]:
print(all_preds)

['Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Review', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Review', 'Question', 'Question', 'Question', 'Question', 'Templates', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Review', 'Question', 'Question', 'Stylus problems', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', 'Question', '

In [9]:
df['predicted_tag'] = all_preds

In [10]:
df.to_csv('./data/unlabelled_df_predicted.csv',index=False)

Examine some of the predicted tags to see how the model performed

In [17]:
df[df['predicted_tag'] == 'Stylus problems']

submission_id  \
63         e05ie7   
188        iifupe   
226        jczozd   
407        nknvw5   

                                                                                       submission_title  \
63                                                        Why is this app so laggy and so buggy lately?   
188                                                             Highlighter behavior prior to v.5.40.30   
226  Why are my strokes so inconsistent? I never had this issue until now. Anyone else with this issue?   
407                                                            Why isn’t there a pencil ✏️ on this app?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                submission_selftext  \
63                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Noticed problems with scrolling and a lot of bugs in the last 2 months, what happened?   
188  Is there a way to user deselect the changes that have been made to the highlighter in version 5.40.30? \n\nThe appearance of the highlighter and dark pages has become awful. The version release notes said changes were made to make the highlight or more visible. I loved the way that the highlighter worked before on dark pages. Now it simply looks awful. I just wanted the strokes to be highlighted not the surrounding area on the page.\n\nThe before version colored only the strokes\nhttps://i.imgur.com/TmE0UHr.jpg\n\n\nThe after version (most recent update) puts the highlight over the text and also colors the area surrounding the text.\nhttps://i.imgur.com/8l8hFHs.jpg\n\n(Pls ignore my terrible handwriting....😬)   
226                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             NaN   
407                                                                                                                                                                                                                                                                                                                                                                                                                                                                   